# Logistic Regression Consulting Project

## Solutions

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [5]:
data = spark.read.csv('/FileStore/tables/customer_churn.csv',inferSchema=True,
                     header=True)

In [6]:
data.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

### Check out the data

In [8]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+

In [9]:
data.columns

Out[ 6 ]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Format for MLlib

We'll ues the numerical columns. We'll include Account Manager because its easy enough, but keep in mind it probably won't be any sort of a signal because the agency mentioned its randomly assigned!

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [13]:
output = assembler.transform(data)

In [14]:
final_data = output.select('features','churn')

In [15]:
final_data.show()

+--------------------+-----+
 features|churn|
+--------------------+-----+
[42.0,11066.8,0.0...| 1|
[41.0,11916.22,0....| 1|
[38.0,12884.75,0....| 1|
[42.0,8010.76,0.0...| 1|
[37.0,9191.58,0.0...| 1|
[48.0,10356.02,0....| 1|
[44.0,11331.58,1....| 1|
[32.0,9885.12,1.0...| 1|
[43.0,14062.6,1.0...| 1|
[40.0,8066.94,1.0...| 1|
[30.0,11575.37,1....| 1|
[45.0,8771.02,1.0...| 1|
[45.0,8988.67,1.0...| 1|
[40.0,8283.32,1.0...| 1|
[41.0,6569.87,1.0...| 1|
[38.0,10494.82,1....| 1|
[45.0,8213.41,1.0...| 1|
[43.0,11226.88,0....| 1|
[53.0,5515.09,0.0...| 1|
[46.0,8046.4,1.0,...| 1|
+--------------------+-----+
only showing top 20 rows

### Test Train Split

In [17]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Fit the model

In [19]:
from pyspark.ml.classification import LogisticRegression

In [20]:
lr_churn = LogisticRegression(labelCol='churn')

In [21]:
fitted_churn_model = lr_churn.fit(train_churn)

In [22]:
training_sum = fitted_churn_model.summary

In [23]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
summary| churn| prediction|
+-------+------------------+-------------------+
 count| 661| 661|
 mean|0.1573373676248109|0.11800302571860817|
 stddev|0.3643942956157553|0.32285601442661266|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+------------------+-------------------+

### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [26]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [27]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[25.0,9672.03,0.0...| 0|[4.45230792137686...|[0.98848255237407...| 0.0|
[26.0,8939.61,0.0...| 0|[6.07794629902272...|[0.99771236456290...| 0.0|
[30.0,10960.52,1....| 0|[2.26207369504999...|[0.90568691128041...| 0.0|
[31.0,5304.6,0.0,...| 0|[3.14100678205269...|[0.95855289785502...| 0.0|
[31.0,5387.75,0.0...| 0|[2.34368907016523...|[0.91243129227933...| 0.0|
[31.0,10058.87,1....| 0|[4.27050553956759...|[0.98621788456223...| 0.0|
[31.0,11743.24,0....| 0|[6.63597066402147...|[0.99868941556546...| 0.0|
[32.0,5756.12,0.0...| 0|[3.95520280896988...|[0.98120522642300...| 0.0|
[32.0,8575.71,0.0...| 0|[3.62025467551138...|[0.97392239377213...| 0.0|
[32.0,8617.98,1.0...| 1|[0.88658296259597...|[0.70818451299529...| 0.0|
[32.0,9885.12,1.0...| 1|[1.68367096055365...|[0.84339001513146...| 0.0|
[32.0,11540.86,0....| 0|[6.61034856653254...|[0.99865544748484...| 0.0|
[32.0,12142.99,0....| 0|[5.56800153402419...|[0.99619641930106...| 0.0|
[33.0,5738.82,0.0...| 0|[4.17581924153855...|[0.98486983782428...| 0.0|
[33.0,7720.61,1.0...| 0|[1.48363135797718...|[0.81512044942693...| 0.0|
[33.0,10709.39,1....| 0|[6.04027912563083...|[0.99762476094077...| 0.0|
[34.0,9401.99,1.0...| 0|[0.91174303165645...|[0.71335670847873...| 0.0|
[34.0,11687.29,0....| 0|[5.72757620020635...|[0.99675560332738...| 0.0|
[34.0,12397.49,0....| 0|[3.17515472159017...|[0.95988852564068...| 0.0|
[34.0,12760.01,0....| 0|[4.89658935674017...|[0.99258339299633...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

### Using AUC

In [29]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [30]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [31]:
auc

Out[ 23 ]: 0.7344559585492229

[Common question - what is a good AUC value?](https://stats.stackexchange.com/questions/113326/what-is-a-good-auc-for-a-precision-recall-curve)

### Predict on brand new unlabeled data

We still need to evaluate the new_customers.csv file!

In [34]:
final_lr_model = lr_churn.fit(final_data)

In [35]:
new_customers = spark.read.csv('/FileStore/tables/new_customers.csv',inferSchema=True,
                              header=True)

In [36]:
new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [37]:
test_new_customers = assembler.transform(new_customers)

In [38]:
test_new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- features: vector (nullable = true)

In [39]:
final_results = final_lr_model.transform(test_new_customers)

In [40]:
final_results.select('Company','prediction').show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+

Ok! That is it! Now we know that we should assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!

## Great Job!